# Deidentify

## Goal: Remove all mention of names in file

Procedure:

1) Collect all unique names from 'Speaker ID' column. 
2) Create lists for all first names, last names, and first+last names. 
3) Create list of pseudonyms with same length as unique names from 'Speaker ID' column.
4) Create dictonary of unique names as keys and assign pseudonyms
5) Replace all entries in 'Speaker ID'.
6) Replace first+last name mentions, first names, and last names in 'Analysis Unit' column.
7) Save revised xslx file.
8) Delete old xlsx file.

In [1]:
import glob
import os
import numpy
import pandas as pd
import random, string

In [2]:
base_dir=os.getcwd()
if os.name == 'nt':
    path_break='\\'
else:
    path_break='/'

In [3]:
# Collects names of files in the '*_Files_to_Deidentify' folders. 
all_files=[]
for file in glob.glob(f'{base_dir}{path_break}Files_to_Deidentify{path_break}*'):
    if '$' not in file:
        all_files.append(file)

In [5]:
for identified_file in all_files:
    id_df = pd.read_excel(identified_file)
    initial_speaker_list = list(set(id_df['Speaker ID'].to_list()))
    speaker_list = [x for x in initial_speaker_list if x == x]

    last_name_list = []
    first_name_list = []
    first_last_name_list = []

    for raw_name in speaker_list:
        if len(raw_name)>0:
            raw_name = raw_name.replace('.', '')
            first_last_name_list.append(raw_name)

            split_name = raw_name.split(' ')
            last_name_list.append(split_name[-1])
            
            if len(split_name) > 2:
                first_two_names = f'{split_name[0]} {split_name[1]}'
                first_name_list.append(first_two_names)
                first_name_list.append(split_name[0]) # Just in case a student has two first names like "Ann Marie" "Ann"
            elif len(split_name) > 1:
                first_name_list.append(split_name[0])

    pseudonym_list = []
    while len(pseudonym_list) != len(first_last_name_list):
        temp_generated = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
        temp_add = []
        temp_add.extend(pseudonym_list)
        temp_add.append(temp_generated)
        temp_add = list(set(temp_add))

        while len(temp_add) == len(pseudonym_list):
            temp_generated = unique_ID = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
            temp_add = []
            temp_add.extend(pseudonym_list)
            temp_add.append(temp_generated)
            temp_add = list(set(temp_add))

        pseudonym_list.append(temp_generated)

In [25]:
# Test inserting images into xlsx files. 
import xlsxwriter
from PIL import Image

test_pic = 'p1.png'

workbook = xlsxwriter.Workbook('pandas_simple.xlsx')
worksheet = workbook.add_worksheet('test_1')

width, height = Image.open(test_pic).size

worksheet.write(0, 0, test_pic)
worksheet.insert_image(0, 0, test_pic)
worksheet.set_row(0, height+10)
worksheet.set_column(0, 0, width/5.3)

workbook.close()

